In [3]:
import pandas as pd
# import plotly.express as px
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

In [4]:
forsyth = gpd.read_file('Geography/Forsyth_CTs.gpkg')
forsyth.crs

forsyth.explore()

In [3]:
new_data = pd.read_csv('NewSales/Forsyth_2023.csv')

new_data['full_address'] = new_data['Address'] + ' Forsyth County, GA'

new_geocoded = pd.read_csv('NewSales/Forsyth_2023_geocoded.csv')

new_full = new_geocoded.merge(new_data, left_on='full_address', right_on='full_address')

# create geodataframe
new_full_gdf = gpd.GeoDataFrame(
    new_full, 
    geometry=gpd.points_from_xy(new_full['long'], new_full['lat']), 
    crs="EPSG:4269"
)

# create the 'year' column
new_full_gdf['year_sale'] = pd.DatetimeIndex(new_full_gdf['Sale Date']).year

# clean up columns
new_full_gdf.rename(columns={
    'Year  Built ':'year_blt',
    'Square Ft ':'Square Ft',
    }, inplace=True)

# create numeric sale column
new_full_gdf['price_number'] = new_full_gdf['Sale Price'].str.replace('[\$,]','',regex=True).str.replace(',','',regex=True).astype(float)

# create price/sf column
new_full_gdf['price_sf'] = new_full_gdf['price_number'] / new_full_gdf['Square Ft']

# spatial join
forsyth_joined = new_full_gdf.sjoin(forsyth, predicate="within")

# only select those sales greater than $1,000, where the size is greater than 75 SF, and includes qualified sales only
forsyth_joined = forsyth_joined[forsyth_joined['price_number'] > 1000]
forsyth_joined = forsyth_joined[forsyth_joined['Square Ft'] > 75]
forsyth_joined = forsyth_joined[forsyth_joined['Qualified Sales'] == 'Qualified']

# final conversions
forsyth_joined['GEOID'] = forsyth_joined['GEOID'].astype(str)
forsyth_joined['unique_ID'] = forsyth_joined['Address'] + '-' + forsyth_joined['Sale Date'].astype(str) + '-' + forsyth_joined['price_number'].astype(str)
forsyth_joined['year'] = pd.DatetimeIndex(forsyth_joined['Sale Date']).year
forsyth_joined['month'] = pd.DatetimeIndex(forsyth_joined['Sale Date']).month
forsyth_joined['year-month'] = forsyth_joined['year'].astype(str) + '-' + forsyth_joined['month'].astype(str)

forsyth_joined = forsyth_joined[[
    'Square Ft',
    'year_sale',
    'year_blt',
    'price_sf',
    'Sale Price',
    'GEOID',
    'Sub_geo',
    'unique_ID', 
    'year', 
    'month', 
    'year-month'
]]

forsyth_joined.columns

Index(['Square Ft', 'year_sale', 'year_blt', 'price_sf', 'Sale Price', 'GEOID',
       'Sub_geo', 'unique_ID', 'year', 'month', 'year-month'],
      dtype='object')

In [4]:
# this is the data to which we'll want to append the new 2023 data
df = pd.read_csv('Geocoded_Final_Joined.csv', thousands=',', keep_default_na=False)
df = df.drop(columns=['field_1'])

df.rename(columns={
        'Year  Built':'year_blt',
        'Year':'year_sale'
    }, inplace=True)

# recast census tract column
df['GEOID'] = df['GEOID'].astype(str)

# create a unique identifier for each sale, which we'll count up later to show Total Sales
df['unique_ID'] = df['Address'] + '-' + df['Sale Date'].astype(str) + '-' + df['price_number'].astype(str)

# create the following columns which will drive the plotly line chart
df['year'] = pd.DatetimeIndex(df['Sale Date']).year
df['month'] = pd.DatetimeIndex(df['Sale Date']).month
df['year-month'] = df['year'].astype(str) + '-' + df['month'].astype(str)


df = df[[
    'Square Ft',
    'year_sale',
    'year_blt',
    'price_sf',
    'Sale Price',
    'GEOID',
    'Sub_geo',
    'unique_ID', 
    'year', 
    'month', 
    'year-month'
]]

df.columns

Index(['Square Ft', 'year_sale', 'year_blt', 'price_sf', 'Sale Price', 'GEOID',
       'Sub_geo', 'unique_ID', 'year', 'month', 'year-month'],
      dtype='object')

In [5]:
df_final = pd.concat([df, forsyth_joined])
# df_final.to_csv('Geocoded_Final_Joined4.csv')

In [19]:
df_final.columns

Index(['Square Ft', 'year_sale', 'year_blt', 'price_sf', 'Sale Price', 'GEOID',
       'Sub_geo', 'unique_ID', 'year', 'month', 'year-month', 'sale_price'],
      dtype='object')

## run a check on the dashboard

In [49]:
df_final = pd.concat([df, forsyth_joined])
df_final['sale_price'] = df_final['Sale Price'].replace('[\$,]', '', regex=True)

# # sub-geography columns - 2
# df_final = df_final[(df_final['Sub_geo'] == 'Cumming') | (df_final['Sub_geo'] == 'South Forsyth')]

# # sub-geography columns - 1
# df_final = df_final[df_final['Sub_geo'] == 'South Forsyth']


year_0 = 2021
year_1 = 2023

df_0 = df_final[df_final['year_sale'] == year_0]
df_1 = df_final[df_final['year_sale'] == year_1]

total_sales_df = df_final[(df_final['year_sale'] >= year_0) & (df_final['year_sale'] <= year_1)]

print(f"Total home sales: {total_sales_df.shape[0]}")
print(f"{year_1} median: {df_1['price_sf'].median()}")
print(f"{year_0} median: {df_0['price_sf'].median()}")
print(f"{((df_1['price_sf'].median() - df_0['price_sf'].median()) / (df_0['price_sf'].median()) * 100)}")

Total home sales: 13365
2023 median: 215.1909036505087
2021 median: 175.92574919999998
22.319162845155986
